# Figure 1, but for Proxima Centauri b.

Overview of the model set-up for the Proxima b case. The *MassFlux* simulation covers the whole sphere, while the *HighRes* simulation is shown by superimposing a high-resolution grid that covers only the substellar region. The graphic shows cloud condensate (white isosurfaces), surface temperature (shading), and free troposphere wind vectors (arrows), focusing on (a) the day side, with the *HighRes* model domain and the cloud condensate isosurface, and (b) the night side of the planet. The cloud condensate is shown using a threshold of $10^{-5}~kg~kg^{-1}$ of total cloud condensate (liquid water plus ice). This figure is available online as an interactive zoom and rotatable figure.

[Skip code and jump to the figure](#Show-the-figure)

----------------------------------

## Import necessary libraries

Standard library

In [1]:
import warnings
from pathlib import Path
import sat_data_to_cubes as wrangler
import os

warnings.filterwarnings("ignore")

3D visualization and mesh analysis library

In [2]:
import pyvista as pv
from pyvista import examples

Scientific stack

In [3]:
import iris
import iris_grib
import numpy as np
import csv
from matplotlib.colors import ListedColormap
# from netCDF4 import Dataset as nc

In [4]:
from aeolus.const import init_const
from aeolus.plot import subplot_label_generator
from aeolus.plot.pv import grid_for_scalar_cube_sph, grid_for_vector_cubes_sph

Set white background for 3d figures.

In [5]:
pv.set_plot_theme("document")

Global definitions.

## Load data

In [6]:
# const = init_const("proxb")
const = init_const("earth")


In [7]:
datadir = os.getcwd() + "/data"
datadir2 = Path.cwd() / "data"
# datadir = "/Users/erose/cira_workspace/unity_workspace/Py2Unity/Assets/SatelliteData/"
datadir = "/Users/erose/cira_workspace/cira-vertical-cloud-products/hdf_practice/files/files_grib/"
#'2022001_lat_long.npz 20220601_cmr_cloud_presence.npz
# print(os.listdir(datadir))

In [8]:
'''
Notes for tomorrow: You have to load in grib data by specifying the variable_id of the connected variable. In the example below, I'm collecting
"Cloud Mixing Ratio" data and "Cloud Ice Mixing-Ratio" data. The corresponding lat/long values should already be included, but if not, I can grab easily.

wanted coord_system(): GeogCS(7160000.0)

Tomorrow we shoud attempt loading in corresponding lat/long data, and then moving forward to understanding how the plotting works with the related data.
Nonetheless, it looks like we can load in ~all~ HRRR data, by just using the sidestep of specifying the variable id. This is great news and means we can load in data
fairly easily, without having to convert it to other means. Woo hoo!

'''
print(datadir)
# dset_global = iris.load(datadir2 / "proxb_global_t_sfc_u_v_w_cld_liq_mf_cld_ice_mf.nc")
dset_conus = iris.load(os.getcwd() + "/data/CONUS_goes16_abi_cloud3d_simple_20220925200117.nc")
# test_data = dset_conus.extract("cloud3d")[0].data
# prev_val = -9999
# for lvl in range(2):
#     for row in range(np.shape(test_data)[1]):
#         for col in range(np.shape(test_data)[2]):
#             if test_data[lvl][row][col] > 0 and test_data[lvl][row][col] != prev_val:
#                 print(test_data[lvl][row][col]); prev_val = test_data[lvl][row][col]
# dset_georing = iris.load(os.getcwd() + "/data/GEO_stitch_cloud3d_16bits_202212201730V2.nc")
# print(dset_georing)
# print(dset_global[0])
dset_ll = np.load(datadir + "cld_data/2022001_lat_long.npz", mmap_mode="r")
dset_cmr = iris_grib.load_cubes(datadir + "20220831hrrr.t23z.wrfprsf00.grib2")#,"cloud_mixing_ratio")#, ["Cloud mixing ratio","Cloud ice mixing-ratio","isobaric","x","y"])
cubes = list(dset_cmr)
lat_proj = dset_ll["arr_0"][0]
lon_proj = dset_ll["arr_0"][1] - 360.0
lat_min = np.min(lat_proj); lat_max = np.max(lat_proj)
lon_min = np.min(lon_proj); lon_max = np.max(lon_proj)
finalLatProj = np.arange(lat_min, lat_max, np.abs(lat_min - lat_max)/1059.0)
finalLonProj = np.arange(lon_min, lon_max, np.abs(lon_min - lon_max)/1799.0)
orig_lat_proj = cubes[0].coord("projection_y_coordinate").points / 17633.33333333
orig_lon_proj = cubes[0].coord("projection_x_coordinate").points / 14983.33333333
print(finalLatProj[0],finalLatProj[-1]); print(np.shape(finalLatProj))
print(finalLonProj[0],finalLonProj[-1]); print(np.shape(finalLonProj))
cmr_cubes = []; icmr_cubes = []
t_sfc_cubes = []; wind_cubes = ["u","v","w"]
for cube in cubes:
    # print(cube)
    cube.coord("projection_y_coordinate").points = finalLatProj #finalLatProj
    cube.coord("projection_x_coordinate").points = finalLonProj #finalLonProj
    ax = str(cube.aux_coords)
    attr = str(cube.attributes["GRIB_PARAM"])
    if attr == "GRIB2:d000c001n022":
        # print(cube.standard_name)
        cmr_cubes.append(cube)
    elif attr == "GRIB2:d000c001n082":
        # print("Cloud ice mixing-ratio")
        cube.units  = "kg kg-1"
        icmr_cubes.append(cube)
    elif attr  == "GRIB2:d000c000n000" and "height" in ax:
        t_sfc_cubes.append(cube)
    elif attr == "GRIB2:d000c002n008" and "<DimCoord: pressure / (Pa)  [50000.]>" in ax: #vertical velocity
        # print("vertical velocity"); print(ax)
        wind_cubes[2] = cube
    elif attr == "GRIB2:d000c002n002" and "<DimCoord: pressure / (Pa)  [50000.]>" in ax: #u component of wind
        # print("u wind"); print(ax)
        wind_cubes[0] = cube
    elif attr == "GRIB2:d000c002n003" and "<DimCoord: pressure / (Pa)  [50000.]>" in ax: #v component of wind
        # print("v wind"); print(ax)
        wind_cubes[1] = cube
    # print(cube.coord("projection_y_coordinate").points)
    # print(cube.coord("projection_x_coordinate").points)
    # print(cube.coord_system())

print("Cloud mixing ratio cubes: ", np.shape(cmr_cubes))
print("Cloud ice mixing-ratio cubes: ", np.shape(icmr_cubes))
print("Temp sfc cubes ", np.shape(t_sfc_cubes))
print("Wind cubes ", np.shape(wind_cubes))
wind_cubes = iris.cube.CubeList(wind_cubes)
# dset_lam = iris.load(datadir / "proxb_lam_cld_mf.nc")

/Users/erose/cira_workspace/cira-vertical-cloud-products/hdf_practice/files/files_grib/
21.13812255859375 52.58593016598381
(1059,)
-134.09547424316406 -60.95788300257675
(1799,)
Cloud mixing ratio cubes:  (40,)
Cloud ice mixing-ratio cubes:  (40,)
Temp sfc cubes  (1,)
Wind cubes  (3,)


## Create `pyvista` objects for a composite plot

Set cloud condensate isosurface threshold.

In [9]:
CC_ISOSURF = [0.00003,0.00004,0.00005,0.00006]  # [kg kg-1]
RADIUS = float(const.radius.data)  # Planet radius [m]
RADIUS

6371200.0

Set viewpoints.

In [10]:
CAM_POS_LATS = [21, 53]
CAM_POS_LONS = [-135, -59]

# CAM_POS_LATS = [10, 20]
# CAM_POS_LONS = [30, 120]

### Global

Set common visualisation propeties.

In [11]:
# z_scale = 75.0
# z_offset = RADIUS * 1.005

z_scale = 100.0
z_offset = RADIUS * 1.01

Create a grid for the surface temperature.

In [12]:
# t_sfc = dset_global.extract_cube("cloud_mixing_ratio")
# print(t_sfc.data)
#Lat/Long points are structured as two 1D arrays of lats and longs of different sizes (144 - lats, 192 - lons)
# lats = t_sfc.coord("latitude").points
# lons = t_sfc.coord("longitude").points
# print("shape = " + str(np.shape(lats)) + str(np.shape(lons)))
# print("lats", lats); print("lons", lons)
# longs = t_sfc.extract("longitude")
# print(lats.data)
# grid_sfc = grid_for_scalar_cube_sph(cmr_cubes[35], z_offset=z_offset, label="global_sfc_grid", isHrrr=True)
# print(t_sfc_cubes[0].data)
# print(t_sfc_cubes[0].coord("projection_y_coordinate").points)
t_sfc_cubes[0].coord("projection_y_coordinate").points = orig_lat_proj
t_sfc_cubes[0].coord("projection_x_coordinate").points = orig_lon_proj
grid_sfc = grid_for_scalar_cube_sph(t_sfc_cubes[0], z_offset=z_offset, label="global_sfc_grid", isHrrr=True)
# print(grid_sfc)
# grid_sfc.texture_map_to_sphere(inplace=True)
# grid_sfc.t_coords = np.zeros((grid_sfc.points.shape[0], 2))
# print(cmr_cubes[0].coord("projection_x_coordinate").points)

Extract 3 wind components from the global model output.

In [ ]:
wind_levels = [5000]  # [m]

winds = dset_global.extract(
    iris.Constraint(level_height=lambda x: x in wind_levels)
).extract(["x_wind", "y_wind", "upward_air_velocity"])

Create a grid for the wind vectors.

In [13]:
grid_vec = grid_for_vector_cubes_sph(
    *wind_cubes,
    vector_scale=RADIUS * 0.004,
    vertical_wind_scale=1e2,
    z_scale=z_scale,
    z_offset=z_offset,
    xstride=1,
    ystride=1,
    label="global_wind_vectors_grid",
    isHrrr=True,
)

In [14]:
grid_vec

StructuredGrid (0x1a17d9d20)
  N Cells:	1902284
  N Points:	1905141
  X Bounds:	-7.422e+06, 5.178e+06
  Y Bounds:	-1.067e+07, -4.990e+06
  Z Bounds:	4.124e+06, 9.082e+06
  Dimensions:	1059, 1799, 1
  N Arrays:	1

Create glyph objects to show wind vectors.

In [15]:
glyphs = grid_vec.glyph(
    orient="global_wind_vectors_grid",
    scale="global_wind_vectors_grid",
    tolerance=0.035,
)

Extract the total cloud condensate.

In [13]:
# cloud_types = ["0001","0010","0011","0100"]
# # global_qct = sum(
# #     dset_georing.extract("cloud3d")
# # )
cloud_cube = dset_conus.extract("cloud3d")[0]
cloud_cube.data = wrangler.testNewMask(cloud_cube.data)
print(cloud_cube.data[cloud_cube.data == 0.00005])
# print(cloud_cube.data)
# print(cloud_cube);print(cloud_cube.data)
# global_qct = sum(cloud_cube)
# # global_qct = sum(iris.cube.CubeList(cmr_cubes)) + sum(iris.cube.CubeList(icmr_cubes))
# lats = dset_georing.extract("latitude")
# lons = dset_georing.extract("longitude")
# lvls = dset_georing.extract("altitude")
lats = np.arange(lat_min, lat_max, np.abs(lat_min - lat_max)/1350.0)
lons = np.arange(lon_min, lon_max, np.abs(lon_min - lon_max)/3250.0)
lvls = [float(i) * 650 for i in range(51)]
lat_coord = iris.coords.DimCoord(lats, "latitude", units="degrees")
lon_coord = iris.coords.DimCoord(lons, "longitude", units="degrees")
lvl_coord = iris.coords.DimCoord(lvls, "altitude", units="km")

cloud_cube.add_dim_coord(lvl_coord,data_dim=0)
cloud_cube.add_dim_coord(lat_coord,data_dim=1)
cloud_cube.add_dim_coord(lon_coord,data_dim=2)
# print(cloud_cube)
# print(cloud_cube.data); print(cloud_cube.data.shape)
# global_qct.add_dim_coord(lvl_coord,data_dim=0)
# global_qct.add_dim_coord(lat_coord,data_dim=1)
# global_qct.add_dim_coord(lon_coord,data_dim=2)
# print(global_qct)

# summed = np.zeros((40,1059,1799),dtype="float32")
# for cube in range(len(cmr_cubes)):
#     try:
#         sum_data = np.zeros((1059,1799),dtype="float32")
#         sum_data += cmr_cubes[cube].data
#         sum_data += icmr_cubes[cube].data
#         summed[cube] = sum_data
#         # print(cube.data)
#     except Exception as e:
#         print(e)
# total_cmr_cubes = cmr_cubes
# for lvl in range(len(summed)):
#     total_cmr_cubes[lvl].data = summed[lvl]
# total_cmr_cubes = iris.cube.CubeList(total_cmr_cubes)
# cmr_single_src = total_cmr_cubes.merge_cube()
# print(cmr_single_src.coord("projection_y_coordinate").points)
# print(cmr_single_src.coord("projection_x_coordinate").points)
# print(cmr_single_src)

HDF5-DIAG: Error detected in HDF5 (1.14.0) thread 1:
  #000: H5A.c line 679 in H5Aopen_by_name(): unable to synchronously open attribute
    major: Attribute
    minor: Can't open object
  #001: H5A.c line 641 in H5A__open_by_name_api_common(): unable to open attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Can't open object
  #002: H5A.c line 464 in H5A__open_common(): unable to open attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Can't open object
  #003: H5VLcallback.c line 1138 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #004: H5VLcallback.c line 1105 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_attr.c line 161 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #006: H5Aint.c line 658 in H5A__open_by_name(): unable 

[5.e-05 5.e-05 5.e-05 ... 5.e-05 5.e-05 5.e-05]


Create a grid for the cloud condensate and extract a 3D contour (isosurface).

In [14]:
# cmr_cubeList = iris.cube.CubeList(total_cmr_cubes)
# print(total_cmr_cubes)
cmr_data = cloud_cube
print(cloud_cube.data[cloud_cube.data > 0])
# cmr_data = cmr_single_src
# cmr_data = global_qct
# print(cmr_data.data)
global_qct_cntr = (
    grid_for_scalar_cube_sph(
        cmr_data, z_scale=z_scale, z_offset=z_offset, label="global_qct_grid", isGeo=True
    )
    .cell_data_to_point_data()
    .contour(isosurfaces=CC_ISOSURF,compute_scalars=True)
    # .contour(isosurfaces=CC_ISOSURF,scalars=cloud_cube.data)
)
print(global_qct_cntr)

[5.e-05 5.e-05 5.e-05 ... 4.e-05 4.e-05 4.e-05]
PolyData (0x19f7146a0)
  N Cells:	28977416
  N Points:	14527847
  X Bounds:	-5.448e+06, 4.402e+06
  Y Bounds:	-9.042e+06, -2.837e+06
  Z Bounds:	2.308e+06, 7.695e+06
  N Arrays:	1



### HighRes

Set common visualisation propeties.

In [15]:
# z_scale = 50.0
# z_offset = RADIUS * .89

z_scale = 120.0
z_offset = RADIUS * 1.02
# TOPLEV = 60
# DLEV = 2  # use every 2nd level
# DY = 3  # stride along y-coordinate
# DX = 3  # stride along x-coordinate

Extract the total cloud condensate from the *HighRes* simulation.

In [16]:
# lam_qct = dset_lam.extract_cube(
#     "mass_fraction_of_cloud_condensed_water_in_air",
# )
# lam_qct = cmr_single_src
lam_qct = cloud_cube
print(lam_qct.data[lam_qct.data == 0.00005])

[5.e-05 5.e-05 5.e-05 ... 5.e-05 5.e-05 5.e-05]


Create a grid from the 3D total cloud condensate field.

In [17]:
lam_grid = grid_for_scalar_cube_sph(
    lam_qct,
    z_scale=z_scale,
    z_offset=z_offset,
    label="lam_qct_grid", isGeo=True
)
print(lam_grid)

StructuredGrid (0x19fea4d00)
  N Cells:	223762500
  N Points:	228389252
  X Bounds:	-6.776e+06, 4.731e+06
  Y Bounds:	-9.736e+06, -2.817e+06
  Z Bounds:	2.328e+06, 8.292e+06
  Dimensions:	1351, 3251, 52
  N Arrays:	1



Create an isosurface of the cloud condensate.

In [18]:
# cntr_water = lam_grid.contour(isosurfaces=[CC_ISOSURF[0]],compute_scalars=True)
# cntr_super = lam_grid.contour(isosurfaces=[CC_ISOSURF[1]],compute_scalars=True)
# cntr_mixed = lam_grid.contour(isosurfaces=[CC_ISOSURF[2]],compute_scalars=True)
# cntr_ice = lam_grid.contour(isosurfaces=[CC_ISOSURF[3]],compute_scalars=True)

cntr_all = lam_grid.cell_data_to_point_data(pass_cell_data=True).contour(isosurfaces=CC_ISOSURF,compute_scalars=True)
# cntr_super = lam_grid.cell_data_to_point_data(pass_cell_data=True).contour(isosurfaces=[CC_ISOSURF[1]],compute_scalars=True)
# cntr_mixed = lam_grid.cell_data_to_point_data(pass_cell_data=True).contour(isosurfaces=[CC_ISOSURF[2]],compute_scalars=True)
# cntr_ice = lam_grid.cell_data_to_point_data(pass_cell_data=True).contour(isosurfaces=[CC_ISOSURF[3]],compute_scalars=True)


Extract a mesh from the HighRes data.

In [19]:
# print(lam_grid.cell_data_to_point_data())
print(cntr_all.cell_data.active_scalars)
# print(cntr_super.cell_data.active_scalars)
# print(cntr_mixed)
# print(cntr_ice)
lam_dom = (
    grid_for_scalar_cube_sph(
        lam_qct[:, ::7, ::7],
        z_scale=z_scale,
        z_offset=z_offset,
        label="lam_dom", isGeo=True
    )
    .extract_geometry()
    .extract_all_edges()
)

[0.e+00 5.e-05 5.e-05 ... 5.e-05 5.e-05 0.e+00]


2023-04-21 12:36:12.620 ( 358.267s) [           207BA]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2023-04-21 12:36:12.807 ( 358.453s) [           207BA]    vtkExtractEdges.cxx:551   INFO| Created 493212 edges


# CLFOS Implementation

In [30]:
start_point = []; end_point = []
with open(os.getcwd() + "/data/3DCloud_CFLOS_P2P_LOS.csv") as cflos_file:
    cflos_list =  list(csv.reader(cflos_file))
    header = cflos_list[0]
    first_point = cflos_list[1]
    last_point = cflos_list[-1]
    # face_node = lam_qct_cntr.mesh.face_node_connectivity
    # indices = face_node.indices_by_location()
    start_point = [float(first_point[3]),float(first_point[5]),float(first_point[5])]
    end_point = [float(last_point[3]),float(last_point[5]),float(last_point[5])]
    # start_lat = cloud_cube.coord("latitude").points[np.argmin(np.abs(float(first_point[0]) -  cloud_cube.coord("latitude").points))]
    # start_lon = cloud_cube.coord("longitude").points[np.argmin(np.abs(float(first_point[1]) -  cloud_cube.coord("longitude").points))]
    # start_alt = cloud_cube.coord("altitude").points[np.argmin(np.abs(((float(first_point[2]) / 100) * 650) -  cloud_cube.coord("altitude").points))]

    # start_idx = lam_qct_cntr.find_closest_point(start_point)
    # end_idx = lam_qct_cntr.find_closest_point(end_point)
    # start_point = lam_qct_cntr.points[start_idx]
    # end_point = lam_qct_cntr.points[end_idx]

    start_idx = cntr_all.find_closest_point(start_point)
    end_idx = cntr_all.find_closest_point(end_point)
    print(cntr_all.points.shape)
    print(start_idx, end_idx)
    start_point = cntr_all.points[start_idx]
    end_point = cntr_all.points[end_idx]
    # start_point[2] *= 2
    # end_point[2] = start_point[2] * 2

    # end_lat = cloud_cube.coord("latitude").points[np.argmin(np.abs(float(last_point[0]) -  cloud_cube.coord("latitude").points))]
    # end_lon = cloud_cube.coord("longitude").points[np.argmin(np.abs(float(last_point[1]) -  cloud_cube.coord("longitude").points))]
    # end_alt = cloud_cube.coord("altitude").points[np.argmin(np.abs(((float(last_point[2]) / 100) * 650) -  cloud_cube.coord("altitude").points))]
    # end_point = [end_lat,end_lon,end_alt]
    # print(header); print(first_point); print(last_point)
# start_point[1] = 0; end_point[1] = 0
end_point = [end_point[0],end_point[1],end_point[2] * 2]
print(start_point); print(end_point)
# points, ind = lam_qct_cntr.ray_trace(start_point,end_point)
# points, ind = global_qct_cntr.ray_trace(start_point,end_point)
points, ind = cntr_all.ray_trace(start_point,end_point)
print(ind)
intersection = pv.PolyData(points)
cflos = pv.Line(start_point,end_point)
# print(global_qct_cntr.points)


(14527847, 3)
11725930 11725930
[ 1916772.42238925 -3463798.73948788  5143819.39138018]
[1916772.4223892507, -3463798.73948788, 10287638.782760367]
[23457506 23460179 23460176]


## Prepare data for plotting

Pack grids and plotting arguments into a single list.

In [35]:
# scalars = np.linalg.norm(lam_qct.points - lam_qct.center, axis=1)
# print(scalars)
# earth_mesh = examples.load_globe()
# earth_mesh = earth_mesh.scale([0.005,0.005,0.005],inplace=True)
earth_mesh = pv.Sphere(radius=RADIUS)
# earth.texture_map_to_sphere(inplace=True)
# earth.textures['2k_earth_daymap'] = examples.load_globe_texture()
annotations = {
    0.00003: "Water",
    0.00004: "Supercooled",
    0.00005: "Mixed",
    0.00006: "Ice",
}
cntr_all["Cloud Type"] = cntr_all.cell_data.active_scalars
print(cntr_all["Cloud Type"].shape)
print(cntr_all["Cloud Type"][cntr_all["Cloud Type"] > 0])
# cntr_water["Cloud Type"] = cntr_water.cell_data.active_scalars
# cntr_super["Cloud Type"] = cntr_super.cell_data.active_scalars
# cntr_mixed["Cloud Type"] = cntr_mixed.cell_data.active_scalars
# cntr_ice["Cloud Type"] = cntr_ice.cell_data.active_scalars

VIS_CONTAINER = [
    {
        # Global surface temperature
        "mesh": earth_mesh,
        "kwargs": {
            "color": "tan",
            "show_scalar_bar": False,
            "smooth_shading": True,
        },
    },
    # {
    #     # Global surface temperature
    #     "mesh": grid_sfc,
    #     "kwargs": {
    #         "cmap": "plasma",
    #         "clim": [240,330],
    #         "show_scalar_bar": False,
    #         "smooth_shading": True,
    #     },
    # },
    {
        # Global surface temperature
        "mesh": intersection,
        "kwargs": {
            "color": "maroon",
            "point_size": 25,
            "label": "Intersection Point",
        },
    },
    {
        # Global surface temperature
        "mesh": cflos,
        "kwargs": {
            "color": "red",
            "line_width": 10,
            "label": "CFLOS Segment",
        },
    },
    # {
    #     # Global wind vectors
    #     "mesh": glyphs,
    #     "kwargs": {"cmap": "Greys", "scalars": "GlyphScale", "show_scalar_bar": False},
    # },
    # {
    #     # Global cloud condensate
    #     "mesh": global_qct_cntr,
    #     "kwargs": {
    #         "cmap": my_colormap,
    #         "clim": [0.00003, 0.00006],
    #         "opacity": 0.5,
    #         "show_scalar_bar": False,
    #         "smooth_shading": True,
    #     },
    # },
    # {
    #     # HighRes cloud condensate
    #     "mesh": lam_qct_cntr,
    #     "kwargs": {
    #         "color": "#ebebff",
    #         "clim": [0.00003,0.00006],
    #         "opacity": 1.0,
    #         "show_scalar_bar": False,
    #         "specular": 0.5,
    #         "ambient": 0.5,
    #         "smooth_shading": True,
    #     },
    # },
    {
        # HighRes cloud condensate
        "mesh": cntr_all,
        "kwargs": {
            "cmap": "jet",
            "n_colors": 4,
            "scalars": "Cloud Type",
            "clim": [0.00003,0.00006],
            "opacity": 1.0,
            "show_scalar_bar": True,
            "specular": 0.5,
            "ambient": 0.5,
            "smooth_shading": True,
            "annotations": annotations,
            "below_color": 'brown',
            "above_color": 'black',
        },
    },
    # {
    #     # HighRes cloud condensate
    #     "mesh": cntr_super,
    #     "kwargs": {
    #         "cmap": "Set3",
    #         "n_colors": 4,
    #         "scalars": cntr_super.cell_data.active_scalars,
    #         "clim": [0.00003,0.00006],
    #         "opacity": 1.0,
    #         "show_scalar_bar": True,
    #         "specular": 0.5,
    #         "ambient": 0.5,
    #         "smooth_shading": True,
    #     },
    # },
    # {
    #     # HighRes cloud condensate
    #     "mesh": cntr_mixed,
    #     "kwargs": {
    #         "color": "paleturquoise",
    #         "clim": [0.00005, 0.00005],
    #         "opacity": 1.0,
    #         "show_scalar_bar": True,
    #         "specular": 0.5,
    #         "ambient": 0.5,
    #         "smooth_shading": True,
    #     },
    # },
    # {
    #     # HighRes cloud condensate
    #     "mesh": cntr_ice,
    #     "kwargs": {
    #         "color": "peru",
    #         "clim": [0.00006, 0.00006],
    #         "opacity": 1.0,
    #         "show_scalar_bar": True,
    #         "specular": 0.5,
    #         "ambient": 0.5,
    #         "smooth_shading": True,
    #     },
    # },
    # {
    #     # HighRes domain edges
    #     "mesh": lam_dom,
    #     "kwargs": {
    #         "style": "wireframe",
    #         "color": "k",
    #         "opacity": 0.2,
    #         "smooth_shading": True,
    #     },
    # },
]

(28977416,)
[5.e-05 5.e-05 5.e-05 ... 5.e-05 5.e-05 5.e-05]


## Plot the results

Assemble the plot.

In [36]:
iletters = subplot_label_generator()
# pv.global_theme.interactive = True
pv.set_jupyter_backend("ipyvtklink")
p = pv.Plotter(shape=(1, len(CAM_POS_LONS)), window_size=np.array([1024, 768 // 2]) * 2)
# p.set_environment_texture(earthmap)
# sphere.plot(texture=tex, smooth_shading=False, background='black', show_axes=False)

# with stars
last_mesh = len(VIS_CONTAINER) - 1
for idx, (cam_lon, cam_lat) in enumerate(zip(CAM_POS_LONS, CAM_POS_LATS)):
    p.subplot(0, idx)
    # p.add_background_image('stars.jpeg')

    p.add_text(f"({next(iletters)})", font="times", font_size=24)
    curr_mesh = 0
    for plot_dict in VIS_CONTAINER:
        # print(plot_dict)
        p.add_mesh(plot_dict["mesh"], **plot_dict["kwargs"])
        if curr_mesh == last_mesh:
            p.add_legend()
        curr_mesh += 1
    p.set_position(pv.grid_from_sph_coords([cam_lon], [90 - cam_lat], [4.5e7]).points)
    # p.enable_point_picking()
    p.set_focus((0, 0, 0))
    p.set_viewup((0, 0, 1))

## Show the figure

In [37]:
# p.show()
p.show(jupyter_backend="ipyvtklink",return_viewer=True)#jupyter_backend="static")# jupyter_backend="ipyvtklink",return_viewer=True
# p.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=2048)